### This script creates a OAAT ensemble of parameter files, reading from a csv file provided and using a pre-defined base file. 

##### In this script we:
1) read in the csv file and make it into a data frame (df)
2) Modify the base parameter file with whatever common modifications we want all of the ensemble members to possess
3) Create and modify minimum and maximum files for each parameter.
4) There is different logic for those which have +- 50 or 100% modification, those that have defined min and maxes and those that have PFT specific ranges, which are defined in their own pft specific parmeter file


In [1]:
import shutil
import os
import netCDF4 as nc
import pandas as pd

##### Control variables and paths 

In [44]:
# Averaged SP calibrated values, from paraemter_averaging and paraemeter_merge scripts. 
averaged_SP_params_files = '../intermediate_pfiles/fates_params_vertsc_dayl_SPcal2.nc' 

# Where to find the list of parameters and their modifications
parameter_list_file = '../csvfiles_for_parameter_modification/NOCOMP_PPE_parameters.csv'
pft_parameter_list = '../csvfiles_for_parameter_modification/pft_specific_parameter_inputs/PFT_params.csv'

# Adjusted default x calibration file to make a reasonable baseline. 
modified_baseline_param_files='../intermediate_pfiles/modified_average_file2.nc'

# What is the path to the new parameter file ensemble 
pdirroot='../paramfiles/NOCOMP_OAAT/nocomp_oaat_v6_'


#### Read the main parameter list CSV file into a pandas DataFrame

In [30]:
print(parameter_list_file)
df = pd.read_csv(parameter_list_file)
fates_parameter_list = df['fates_parameter'].tolist()
minp = df['min'].tolist() 
maxp = df['max'].tolist() 
dimp = df['dim'].tolist()

../csvfiles_for_parameter_modification/NOCOMP_PPE_parameters.csv


#### Read the PFT parameter list CSV file into a pandas DataFrame

In [32]:
print(pft_parameter_list)
dfp = pd.read_csv(pft_parameter_list)
pftindex = dfp['pft_index'].tolist()
pftparam = dfp['param'].tolist() 
pftmin = dfp['min'].tolist()
pftmax = dfp['max'].tolist()

../csvfiles_for_parameter_modification/pft_specific_parameter_inputs/PFT_params.csv


#### map from the variables in the pftvariable csv file into the fates pft space

In [34]:
string_mapping = {
    "fates_rad_stem_rhovis": "rhosvis",
    "fates_rad_stem_rhonir": "rhosnir",
    "fates_rad_leaf_rhonir": "rholnir",
    "fates_rad_leaf_rhovis": "rholvis",
    "fates_rad_leaf_taunir": "taulnir",
    "fates_rad_leaf_tauvis": "taulvis",
    "fates_leaf_stomatal_slope_medlyn": "medlynslope",
    "fates_leaf_slatop": "sla",
    "fates_rad_leaf_xl": "xl"   
}

#### Modify average file for NOCOMP specific baseline modifications


In [46]:
shutil.copy(averaged_SP_params_files, modified_baseline_param_files)
nc_file = nc.Dataset(modified_baseline_param_files, 'r+')  # Open the file in write mode

#atkin = nc_file.variables['fates_maintresp_leaf_atkin2017_baserate'][:]
#atkin_new = atkin
#atkin_new[0] = atkin_new[0]*0.7
#nc_file.variables['fates_maintresp_leaf_atkin2017_baserate'][ :] = atkin_new  # Assign the modified values to the variable

seeds = nc_file.variables['fates_recruit_seed_supplement'][:]
seeds_new = seeds
seeds_new = seeds_new*0+4
nc_file.variables['fates_recruit_seed_supplement'][ :] = seeds_new  # Assign the modified values to the variable

seedg = nc_file.variables['fates_recruit_seed_germination_rate'][:]
seedg_new = seedg
seedg_new = seedg_new*1.5
nc_file.variables['fates_recruit_seed_supplement'][ :] = seedg_new  # Assign the modified values to the variable

initd = nc_file.variables['fates_recruit_init_density'][:]
initd_new = initd
initd_new = initd_new*2
nc_file.variables['fates_recruit_height_min'][ :] = initd_new  # Assign the modified values to the variable

rm = nc_file.variables['fates_maintresp_leaf_model'][:]
rm_new = rm
rm_new = 2
nc_file.variables['fates_maintresp_leaf_model'][ :] = rm_new  # Assign the modified values to the variable


#inith = nc_file.variables['fates_recruit_init_density'][:]
#inith_new = inith
#inith_new = inith_new*1
#nc_file.variables['fates_recruit_height_min'][ :] = inith_new  # Assign the modified values to the variable

cushion = nc_file.variables['fates_alloc_storage_cushion'][:]
cushion_new = cushion
cushion_new = cushion_new*0+2.4
cushion_new[0]=1.2 
cushion_new[3]= 1.2       
nc_file.variables['fates_alloc_storage_cushion'][ :] = cushion_new  # Assign the modified values to the variable

#make FATES CE negative to speed up runs.
fates_comp_excln = nc_file.variables['fates_comp_excln'][:]
cenew = fates_comp_excln *0 -1
nc_file.variables['fates_comp_excln'][ :] = cenew # Assign the modified values to the variable

#make FATES Crad model twostream
fates_rad_model = nc_file.variables['fates_rad_model'][:]
rmnew = fates_rad_model *0 +2
nc_file.variables['fates_rad_model'][:] = rmnew # Assign the modified values to the variable

print(nc_file['fates_rad_model'][:])
print(rmnew)

#hlmmap = nc_file.variables['fates_hlm_pft_map'][:,:]
#hlmnew = hlmmap
#hlmnew[:,13]=0
#hlmnew[:,12]=0
#hlmnew[14, 10] = 1
#hlmnew[15, 11] = 1
#nc_file.variables['fates_hlm_pft_map'][:, :] = hlmnew  # Assign the modified values to the variable

#Iincrease vcmax to make the 
#vcm = nc_file.variables['fates_leaf_vcmax25top'][:,:]
#vcmnew = vcm
#vcmnew[0]=vcmnew[0]*1.2
#nc_file.variables['fates_leaf_vcmax25top'][:, :] = vcmnew  # Assign the modified values to the variable


nc_file.close()

2.0
2.0


In [48]:
nc_file = nc.Dataset(modified_baseline_param_files, 'r+')  

print(nc_file['fates_rad_model'][:])
nc_file.close()

2.0


#### Retreive the full list of the parameters 

In [50]:
reference_vars = nc.Dataset(averaged_SP_params_files, 'r+') 
fatespfts=reference_vars.dimensions['fates_pft']
print(fatespfts.size)

modified_baseline_param_files

12


'../intermediate_pfiles/modified_average_file2.nc'

#### Modify variables in OAAT ensemble according to the values in the input csv fil

In [52]:
counter = 0
import numpy as np
for index, parameter in enumerate(fates_parameter_list):
    if parameter in reference_vars.variables:
        # Perform operations on each parameter
        print('#',index, parameter)  # Replace with your operations
       # print('minp index',minp[index])
#-------------------------------------------------------
        # Make minimum values    
        counter = counter + 1
        newfile = pdirroot + str(counter) + '.nc'    
        shutil.copy(modified_baseline_param_files, newfile)
        nc_file = nc.Dataset(newfile, 'r+')
        variable_to_modify = nc_file.variables[parameter]
            
        if minp[index] == '50percent':
            if variable_to_modify.dtype == int:
                print('integer')
                variable_data = variable_to_modify[:]
                variable_data = (variable_data * 0.5).astype(int)
                variable_to_modify[:] = variable_data  
                #print(variable_data)
            else:
                variable_to_modify[:] *= 0.5    
                print('pft min',variable_to_modify[:])
        elif minp[index] == '100percent':
                variable_to_modify[:] *= 0            
        elif minp[index] ==  'pft':
            shortvar = string_mapping[parameter]
            table = dfp[dfp['param'] == shortvar]
            pftmins=table['min'].tolist()
            variable_to_modify[:] *=  0
           
            for p in range(1, 12 ):               
                variable_to_modify[p-1] += float(pftmins[p-1])
            #variable_to_modify[12] += float(pftmins[10])
            #variable_to_modify[13] += float(pftmins[11])
            print(shortvar, ' table ',pftmins)
            print(shortvar, counter ,'min ',variable_to_modify[:])
        else:
            print('minp',minp[index])
            variable_to_modify[:] *=  0
            variable_to_modify[:] += float(minp[index])
        
        nc_file.close()  # Close the modified file
        
#-------------------------------------------------------
        # Make maximum values    
        counter = counter + 1
        newfile = pdirroot + str(counter) + '.nc'    
        shutil.copy(modified_baseline_param_files, newfile)
        nc_file = nc.Dataset(newfile, 'r+')
        variable_to_modify = nc_file.variables[parameter]
        if minp[index] == '50percent':
            if variable_to_modify.dtype == int:
                print('integer')
                variable_data = variable_to_modify[:]
                variable_data = (variable_data * 1.5).astype(int)
                variable_to_modify[:] = variable_data 
                print(variable_data)
            else:
                variable_to_modify[:] *= 1.5  
                print('pftmax',variable_to_modify[:])
        elif minp[index] == '100percent':
                variable_to_modify[:] = 1 
                print('100%',counter,parameter)
            
        elif minp[index] ==  'pft':
            shortvar = string_mapping[parameter]
            table = dfp[dfp['param'] == shortvar]
            pftmaxs=table['max'].tolist()
            variable_to_modify[:] *=  0
           
            for p in range(1, 12 ):
                variable_to_modify[p-1] += float(pftmaxs[p-1])
            #variable_to_modify[12] += float(pftmaxs[10])
            #variable_to_modify[13] += float(pftmaxs[11])
            print(shortvar, ' table ',pftmaxs)
            print(shortvar, counter, 'max ',variable_to_modify[:])
        else:
            print('maxp',maxp[index])
            variable_to_modify[:] *=  0
            variable_to_modify[:] += float(maxp[index])


        nc_file.close()  # Close the modified file

    else: #doenst exist 
        print('variable is not in file', parameter)

# 0 fates_alloc_storage_cushion
pft min [0.6 1.2 1.2 0.6 1.2 1.2 1.2 1.2 1.2 1.2 1.2 1.2]
pftmax [1.8 3.6 3.6 1.8 3.6 3.6 3.6 3.6 3.6 3.6 3.6 3.6]
# 1 fates_allom_agb_frac
pft min [0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3]
pftmax [0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9]
# 2 fates_allom_agb1
pft min [0.03365    0.0682006  0.01965285 0.13268475 0.03365    0.0364349
 0.03448    0.03448    0.03448    0.005      0.005      0.005     ]
pftmax [0.10095    0.2046018  0.05895855 0.39805425 0.10095    0.1093047
 0.10344    0.10344    0.10344    0.015      0.015      0.015     ]
# 3 fates_allom_agb2
pft min [0.488      0.47245205 0.5436675  0.41606605 0.488      0.51866055
 0.286      0.286      0.286      0.286      0.286      0.286     ]
pftmax [1.464      1.41735615 1.6310025  1.24819815 1.464      1.55598165
 0.858      0.858      0.858      0.858      0.858      0.858     ]
# 4 fates_allom_agb3
pft min [0.97 0.97 0.97 0.97 0.97 0.97 0.97 0.97 0.97 0.97 0.97 0.97]
pftmax [2.91

In [ ]:
nc_file = nc.Dataset(newfile, 'r+')
#print(nc_file.variables['fates_mort_ip_size_senescence'])
print(nc_file.variables['fates_mort_ip_size_senescence'][:])
nc_file.close() 